# Introduction




Them main dataset that will be used throughout this analysis is https://www.kaggle.com/datasets/andrewmvd/steam-reviews/code. This dataset contains 1.5 million reviews from the Steam platform. The reviews are from 2014 to 2019. The dataset contains the following columns:

In [1]:
import pandas as pd

# Part 1: Data Cleaning and Statistics

### Part 1a: Data Description
Them main dataset that will be used throughout this analysis is https://www.kaggle.com/datasets/andrewmvd/steam-reviews/code. This dataset contains 1.5 million reviews from the Steam platform. The reviews are from 2014 to 2019. The dataset contains the following columns: `app_id`, `app_name`, `review_text`, `review_score`, `review_votes`.

* **app_id**: Unique ID of the game
* **app_name**: Name of the game
* **review_text**: Text of the review
* **review_score**: Score of the review (1 is positive, )


In [4]:
review_data.columns

Index(['app_id', 'app_name', 'review_text', 'review_score', 'review_votes'], dtype='object')

In [3]:
review_data = pd.read_csv("dataset.csv")
review_data.head()

,app_id,app_name,review_text,review_score,review_votes
0,10,Counter-Strike,Ruined my life.,1,0
1,10,Counter-Strike,This will be more of a ''my experience with th...,1,1
2,10,Counter-Strike,This game saved my virginity.,1,0
3,10,Counter-Strike,• Do you like original games? • Do you like ga...,1,0
4,10,Counter-Strike,"Easy to learn, hard to master.",1,1


### Part 1b: Data Cleaning

In [ ]:
# Remove games which have less than # reviews such that the game is not given a review by Steam



# (Optional) Remove games which are not #% of the total number of reviews in the dataset
# This is to include only the most popular games in the dataset for less noise


# (Potential Experiment Route) Divide games by genre

### Part 1 Conclusion

# Part 2: Sentiment Analysis Experiments

### Part 2 Conclusion

Best way to do sentiment analysis

# Part 3: 
Using the best sentiment analysis model and using those insights to reach deeper insights about the Steam Reviews dataset